# 進捗状況可視化ノートブック（オフライン）  

本ノートブックでは、収集されたログ情報をグラフ表示するサンプルをいくつか記載しています。  

## パラメータ指定

対象のコース名等を指定します。

In [ ]:
import os

from IPython.display import display
from ipywidgets import widgets
from nbgrader.api import Gradebook, MissingEntry

db_path = os.path.join(os.environ['HOME'], 'nbgrader', os.environ['MOODLECOURSE'], 'gradebook.db')
gb = Gradebook(f'sqlite:///{db_path}',
               os.environ['MOODLECOURSE'])

assignment_selection = widgets.Dropdown(
    options = [d.name for d in gb.assignments],
    description = "課題を選択してください:",
)
display(assignment_selection)

In [ ]:
import os

# Control Panel → Token より発行したトークンを入力してください
TOKEN = 

# -- 以下は必要に応じて変更してください 基本的には変更不要です。--
COURSE = os.environ['MOODLECOURSE']
DB_PATH = os.path.join(os.environ['HOME'], 'nbgrader', COURSE, 'exec_history.db')
ASSIGNMENT = assignment_selection.value

## ログ収集

ログ収集用に稼働しているAPIを利用して、各学生のディレクトリに出力されたログ情報をDBに収集します。  
以下のセルにて、各パラメータを指定してください。指定する項目は以下の通りです。  

* dt_from（Optional）  
  収集対象ログの日時（始点）。ここに指定した日時以降のログのみをDBに登録する。
* dt_to  （Optional）  
  収集対象ログの日時（終点）。ここに指定した日時以前のログのみをDBに登録する。
* assignment  （Optional）  
  収集対象ログの課題名。ここに指定した課題に登録されているノートブックのログのみをDBに登録する。  
  指定しなければ、nbgraderのDBに登録されている、コース内の全ての課題を対象とする。

In [ ]:
from datetime import datetime, timedelta, timezone
import requests

JST = timezone(timedelta(hours=+9), 'JST')

# -- パラメータ指定 --
dt_from = datetime(1970, 1, 1, 0, 0, 0, tzinfo=JST)
dt_to = datetime.now(JST)
headers = {"Content-Type": "application/json",
           "Authorization": f"token {TOKEN}"}
data = {
    'course': COURSE,
    'from': dt_from.isoformat(),
    'to': dt_to.isoformat(),
    'assignment': ASSIGNMENT,
}
r = requests.post(f'https://{os.environ["JUPYTERHUB_FQDN"]}/services/teachertools/api/log_collect',
                  headers=headers,
                 json=data)

print(r.status_code)
print(r.json())

## サンプル

### 1. cell番号ごとの、正常終了したセルの数  

cell番号ごとの、正常終了したセルの数を集計します。  
同一ユーザが同じセルを複数回実行している場合、最新の実行結果のみを集計対象とします。

x軸: cell_id  
y軸: 正常終了数  

In [ ]:
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

sql = """
WITH LatestLog AS (
  SELECT
    cell.id as cell_id,
    cell.assignment as assignment,
    student_id as student_id,
    log_execute_reply_status as log_execute_reply_status,
    CASE log_execute_reply_status
      WHEN 'ok' THEN 1
      ELSE 0
    END as ok_count,
    MAX(log_sequence) AS max_log_sequence
  FROM
    cell left outer join log on cell.id = log.cell_id and cell.assignment = log.assignment
  WHERE
    cell.assignment = ?
GROUP BY
    cell.id,
    cell.assignment,
    student_id,
    log_execute_reply_status
)
SELECT
  cell_id,
  sum(ok_count)
FROM
  LatestLog
group by
  cell_id
;
"""

with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
    cur = con.cursor()
    cur.execute(sql, [ASSIGNMENT])
    result = cur.fetchall()
    print(result)

x = list()
y = list()
for d in result:
    x.append(d[0])
    y.append(d[1])

fig, ax = plt.subplots(figsize=[10, 4.2])
plt.setp(ax.get_xticklabels(), rotation= 45, ha="right") 

# 自動で表示させると、Y軸が小数単位になる場合があるので、整数指定する
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_ylabel('completed cell', fontsize=16)
ax.set_xlabel('meme_id', fontsize=16)
ax.bar(x, y)


### 2. 正常終了したセルの割合/時刻  

指定した時刻間の、時間経過に伴う正常終了したセルの割合の変化を確認します。  

x軸: 時刻  
y軸: 正常終了率  

※ `正常終了率` = 実行結果が`ok`のセル数 / (対象ノートブックの合計セル数*受講者数)  


In [ ]:
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import datetime

start_dt = datetime.datetime.strptime('2025/07/23 15:30:00 +0900', '%Y/%m/%d %H:%M:%S %z')
end_dt = datetime.datetime.strptime('2025/07/23 17:00:00 +0900', '%Y/%m/%d %H:%M:%S %z')
interval = 60 * 10 # 10min

# 課題を解き終わったあとは実行しない＝ログが無い
# 経過時刻ごとに、その時刻以下の中で最新のログ（end < 各時刻）を取る
sql = f"""
WITH RECURSIVE dtime_ranges AS (
    SELECT datetime(?) AS dtime
    UNION ALL
    SELECT datetime(dtime, '+{interval} seconds')
    FROM dtime_ranges
    WHERE dtime < datetime(?, '-{interval} seconds')
)
SELECT
    strftime('%H:%M', dtime_ranges.dtime, '+9 hours') AS dtime,
    log_grouped.assignment,
    COALESCE(SUM(log_grouped.ok_count) * 100 / total_cell.c, 0) AS rate,
    total_cell.c
FROM
    dtime_ranges
LEFT JOIN (
    SELECT
        dtime_ranges.dtime AS dtime,
        log.assignment,
        COUNT(DISTINCT log.student_id || '-' || log.cell_id) AS ok_count
    FROM
        dtime_ranges
    LEFT JOIN log ON
        log.assignment = ?
        AND log.log_execute_reply_status = 'ok'
        AND strftime('%Y-%m-%d %H:%M', log.log_end) <= dtime_ranges.dtime
    GROUP BY
        dtime_ranges.dtime, log.assignment
) log_grouped ON dtime_ranges.dtime = log_grouped.dtime
CROSS JOIN (
    SELECT
        student_count.total * cell_count.total AS c
    FROM
        (SELECT COUNT(*) as total FROM student) student_count,
        (SELECT COUNT(*) as total FROM cell WHERE assignment = ?) cell_count
) total_cell
GROUP BY
    dtime_ranges.dtime, log_grouped.assignment;

"""
# 指定した時刻間で、[ユーザ・セル]ごとに正常終了があれば1, なければ0

with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
    # 列名で結果にアクセスできるようにする
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute(sql, (start_dt, end_dt, ASSIGNMENT, ASSIGNMENT))
    result = cur.fetchall()

x = list()
y = list()
for d in result:
    x.append(d['dtime'])
    y.append(d['rate'])

fig, ax = plt.subplots(figsize=[10, 4.2])
ax.set_ylabel('completed cell rate (%)')
ax.set_xlabel('timestamp (JST)')
ax.set_ylim(0, 100)
ax.plot(x, y)


### 3. セルごとの正常終了人数  

指定した時刻での、全体の進捗を確認します。  
各正常終了率での人数を確認できます。  
x軸=0(%)は全員該当するのでy軸=受講者数、x軸=100(%)でy軸=2(人)であれば、全てのセルが正常終了した学生は2人、というように読み取ります。

x軸: 正常終了率
y軸: 人数

In [ ]:
import datetime

import sqlite3
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

dt_to = datetime.datetime.strptime('2025/07/23 17:15:00 +0900', '%Y/%m/%d %H:%M:%S %z')

sql = """
WITH LatestLog AS (
  SELECT
    cell.id as cell_id,
    cell.assignment as assignment,
    student_id as student_id,
    log_execute_reply_status as log_execute_reply_status,
    CASE log_execute_reply_status
      WHEN 'ok' THEN 1
      ELSE 0
    END as ok_count,
    MAX(log_sequence) AS max_log_sequence
  FROM
    cell left outer join log on cell.id = log.cell_id and cell.assignment = log.assignment
  WHERE
    cell.assignment = ?
    AND log_end < ?
GROUP BY
    cell.id,
    cell.assignment,
    student_id,
    log_execute_reply_status
)
SELECT
  ROW_NUMBER() OVER(ORDER BY cell_id ASC) num,
  cell_id,
  sum(ok_count) AS ok_num,
  student.num AS student_num
FROM
  LatestLog,
  (SELECT COUNT(*) num
   FROM student) AS student
group by
  cell_id
order by
  cell_id
;
"""

with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
    # 列名で結果にアクセスできるようにする
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute(sql, [ASSIGNMENT, dt_to])
    result = cur.fetchall()

# 0個目のセル（実際は存在しない）の位置は全員該当（クリア済）として初期化
x = [0]
y = [result[0]['student_num']]
for d in result:
    x.append(d['num']*100/len(result))
    y.append(d['ok_num'])

fig, ax = plt.subplots(figsize=[10, 4.2])

# 自動で表示させると、Y軸が小数単位になる場合があるので、整数指定する
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_xlim(0, 100)
ax.set_ylim(0, result[0][3])
ax.set_ylabel('number', fontsize=16)
ax.set_xlabel('progress (%)', fontsize=16)
ax.plot(x, y)


### 4. 試行回数/試行時間  

指定したセルの、試行回数と試行時間の関係を確認します。  
試行回数は実行回数、試行時間は各実行履歴間の時間から、離席時間とみなす時間以上のものを除去した合計時間としています。

x軸: 試行回数  
y軸: 試行時間

#### 対象セルIDの特定

以下のセルを実行すると、対象assignmentのノートブックに登録されているセルの一覧を出力します。  
実際のセルを見て設定する場合は、ノートブックの各セルのメタデータを参照してください。

In [ ]:
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import datetime

sql = """
SELECT
  id
  FROM cell
  WHERE assignment = ?
"""
with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
    cur = con.cursor()
    cur.execute(sql, [ASSIGNMENT])
    result = cur.fetchall()
result

#### データ取得・グラフ出力  

cell_idを指定し、「セルを解くのにかかった時間」と「セルを実行した回数」の関係をグラフで表示します。  
指定した時刻間を課題に取り組んだ時間とし、その合計から休憩等で離席したと判断した時間を引いた時間を「セルを解くのにかかった時間」とします。

In [ ]:
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import datetime

dt_from = datetime.datetime.strptime('2025/07/23 16:00:00 +0900', '%Y/%m/%d %H:%M:%S %z')
dt_to = datetime.datetime.strptime('2025/07/23 17:00:00 +0900', '%Y/%m/%d %H:%M:%S %z')
cell_id = '19a2ccfe-6792-11f0-9463-02420a0100a2'
threshold_execution_seconds = 60 * 30 #課題に取り組んでいない（離席中）とみなす時間

sql = """
SELECT
  student_id,
  SUM(working_time) as total_working_seconds,
  COUNT(*) as total_working_quant
  FROM (SELECT
            student_id,
            ifnull(unixepoch(log_end) - unixepoch(LAG(log_end, 1) OVER (PARTITION BY student_id ORDER BY datetime(log_end))), 0) AS working_time,
            log_end
          FROM log
          WHERE log.assignment = ?
          AND   cell_id = ?
          AND   log_end between ? AND ?
          AND   log_execute_reply_status = ?
          ) duration
  WHERE working_time < ?
  GROUP BY
    student_id
"""
with sqlite3.connect(f"file:{DB_PATH}?mode=ro", uri=True) as con:
    # 列名で結果にアクセスできるようにする
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute(sql, [ASSIGNMENT, cell_id,dt_from,dt_to, 'ok', threshold_execution_seconds])
    result = cur.fetchall()

x = list()
y = list()
for d in result:
    x.append(d['total_working_seconds']//60)
    y.append(int(d['total_working_quant']))

fig, ax = plt.subplots(figsize=[10, 4.2])
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_xlabel('trial required (min)')
ax.set_ylabel('trial number (count)')
ax.set_xlim(0, max(x)+1)
ax.bar(x, y)
